In [47]:
# noinspection JupyterPackage
from collections import Counter
import envs.twitter as credentials
import datetime
import tweepy
import random
import sys
import json
import pandas as pd

#set seed
random.seed(789)
random_min=20
random_max=100
hashtags=[
    "VacunaCOVID19",
    "sismo",
    "WandaVision",
    "UKlockdown3",
    "Indonesia",
    "iPhone",
    "CES2021",
    "Adictosdigitales",
    "Tech",
    "innovation",
    "iphone",
    "EE.UU"]
days_ago=5
#default date_until to today
date_until=datetime.date.today()
#default date_since to N (days_ago) from today
date_since=date_until - datetime.timedelta(days=days_ago)
exclude_filter="retweets"

In [48]:
#generate random number, arguments used as limits
def random_generator(min_arg=random_min,max_arg=random_max):
    try:
        random_result = random.randint(min_arg,max_arg)
        print(random_result)
        return random_result
    except Exception:
        print("Oops!", sys.exc_info()[0], "occurred.")

tweets_needed=random_generator()

81


In [49]:
def get_tweets(
        hashtag_arg,
        count_arg=100,
        tweets_arg=tweets_needed,
        date_since_arg=date_since,
        date_until_arg=date_until,
        exclude_filter_arg=exclude_filter
):
    try:
        # Authentication object creation - read only
        auth = tweepy.AppAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
        # API object creation
        api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
        # Cursor search with filters
        tweets = tweepy.Cursor(api.search,
            q="#{} since:{} until:{} -filter:{}".format(hashtag_arg,date_since_arg,date_until_arg,exclude_filter_arg),
            count=count_arg
        ).items(tweets_needed)
        return tweets
    except Exception:
        print("Oops!", sys.exc_info()[0], "occurred.")

In [50]:
#extract tweet specific information argument should be tweet object
def extract_tweet(tweet_arg):
    extract=[
        tweet_arg.id,
        tweet_arg.created_at.strftime('%m/%d/%Y'),
        tweet_arg.favorite_count,
        tweet_arg.retweet_count,
        tweet_arg.lang,
        tweet_arg.text,
        tweet_arg.author.id,
        tweet_arg.author.name,
        tweet_arg.author.screen_name,
        tweet_arg.author.verified,
        tweet_arg.author.followers_count
    ]
    return extract

In [51]:
# extract required information, argument should be tweepy.models.SearchResults object
def extract_info(results_arg):
    extract_result=[]
    for tweet in results_arg:
       extract_result.append(extract_tweet(tweet))
    return extract_result

In [52]:
#create dictionary of tweets iterating thru all hastags in hastags list
dictOfTweets = { i : extract_info(get_tweets(i,date_since_arg="2021-02-02",date_until_arg="2021-02-07")) for i in hashtags }

In [53]:
#function that creates dataframes with specific cols, df based on hashtag. argument should be string with no #
def df_create(hashtag_arg):
    return pd.DataFrame(dictOfTweets[hashtag_arg],
                  columns = [
                      'id', 'date_dreated','favorite','retweet','language','content',
                      'author_id','author_name','author_screen_name','verified','followers'
                  ]
                  )

1.	Propuesta de cómo medir la influencia de un usuario de la muestra.(2 puntos)


La propuesta para medir la influencia:
Influencia = estimado_impportancia_retweets+estimado_importancia_favoritos+estimado_cantidad_tweets

In [54]:
def influence_calculator(df_arg):
    df_arg['tweets_estimate'] = df_arg.groupby(['author_id'])['followers'].transform('count')/100
    df_arg["favorite_estimate"]=(10*df_arg["favorite"])/df_arg["favorite"].max()
    df_arg["retweet_estimate"]=(10*df_arg["retweet"])/df_arg["retweet"].max()
    df_arg["influence"]=df_arg["tweets_estimate"]+df_arg["favorite_estimate"]+df_arg["retweet_estimate"]
    df_arg.sort_values(by=['influence'], inplace=True,ascending=False)
    influence=df_arg[["id","author_screen_name","influence"]].head(5)
    return influence